<a href="https://colab.research.google.com/github/mandarina125/Desaf-o_manipulando-_una_base_de_datos/blob/main/Desaf%C3%ADo_manipulando_una_base_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Crear la base de datos local con la biblioteca SQLAlchemy.

In [25]:
import sqlalchemy

In [26]:
from sqlalchemy import create_engine, MetaData, Table, inspect, text

In [36]:
engine = create_engine('sqlite:///:memory: ')

In [38]:
import pandas as pd

Escribir los datos del archivo CSV en esta base de datos local.

In [29]:
archivo = "clientes_banco.csv"

In [30]:
datos = pd.read_csv(archivo)

In [39]:
datos.to_sql("clientes", engine, if_exists="replace", index=False)


438463

In [42]:
print (inspect(engine).get_table_names())

['clientes']


Inspeccionar las primeras 5 lineas

In [43]:
datos

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
0,5008804,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
1,5008805,32,Nivel superior,Relación-estable,2,Empleado,Otro,12,427500.0,1,Departamento alquilado
2,5008806,58,Nivel intermedio,Casado,2,Empleado,Seguridad,3,112500.0,1,Casa/Departamento propio
3,5008808,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
4,5008809,52,Nivel intermedio,Soltero,1,Business Partner,Ventas,8,270000.0,0,Casa/Departamento propio
...,...,...,...,...,...,...,...,...,...,...,...
438458,6840104,62,Nivel intermedio,Divorciado,1,Pensionista,Otro,0,135000.0,0,Casa/Departamento propio
438459,6840222,43,Nivel intermedio,Soltero,1,Empleado,Construcción Civil,8,103500.0,0,Casa/Departamento propio
438460,6841878,22,Nivel superior,Soltero,1,Business Partner,Ventas,1,54000.0,0,Vive con los padres
438461,6842765,59,Nivel intermedio,Casado,2,Pensionista,Otro,0,72000.0,0,Casa/Departamento propio


**Realizar tres actualizaciones en la base de datos:**

Actualizar el registro del cliente ID 6840104 cuyo rendimiento anual cambió a 300000.

In [44]:
from sqlalchemy.exc import SQLAlchemyError
query = "UPDATE clientes SET Rendimiento_anual = 300000 WHERE ID_Cliente = 6840104"

try:
  with engine.connect() as connection:
    r_set = connection.execute(text(query))
    connection.commit() # Commit the transaction

except SQLAlchemyError as e:
  print(e)

else:
  print("Actualización exitosa: ", r_set.rowcount)

Actualización exitosa:  1


In [46]:
with engine.connect() as conn:
    resultado = conn.execute(text("SELECT * FROM clientes WHERE ID_Cliente = 6840104")).fetchone()
    print(resultado)


(6840104, 62, 'Nivel intermedio', 'Divorciado', 1, 'Pensionista', 'Otro', 0, 300000.0, 0, 'Casa/Departamento propio')


In [47]:
# Volver a cargar la tabla desde SQLite
df_actualizado = pd.read_sql("SELECT * FROM clientes", engine)

# Verificar el cliente actualizado
df_actualizado[df_actualizado["ID_Cliente"] == 6840104]


,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
438458,6840104,62,Nivel intermedio,Divorciado,1,Pensionista,Otro,0,300000.0,0,Casa/Departamento propio


Eliminar el registro de cliente ID 5008809, ya que esta persona ya no tiene cuenta en la institución financiera.

In [49]:
from sqlalchemy.exc import SQLAlchemyError
query = "DELETE FROM clientes WHERE ID_Cliente = 5008809"

try:
  with engine.connect() as connection:
    r_set = connection.execute(text(query))
    connection.commit() # Commit the transaction

except SQLAlchemyError as e:
  print(e)

else:
  print("Registro borrado: ", r_set.rowcount)



Registro borrado:  1


In [50]:
with engine.connect() as conn:
    resultado = conn.execute(text("SELECT * FROM clientes WHERE ID_Cliente = 5008809")).fetchone()
    print(resultado)


None


In [51]:
# Volver a cargar la tabla desde SQLite
df_actualizado = pd.read_sql("SELECT * FROM clientes", engine)

# Verificar el cliente eliminado
df_actualizado[df_actualizado["ID_Cliente"] == 5008809]

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda


Crear un nuevo registro de cliente siguiendo las especificaciones a continuación

In [52]:
from sqlalchemy import text
from sqlalchemy.exc import SQLAlchemyError

insert_query = """
INSERT INTO clientes (
    ID_Cliente, Edad, Grado_estudio, Estado_civil, Tamaño_familia,
    Categoria_de_renta, Ocupacion, Años_empleado, Rendimiento_anual,
    Tiene_carro, Vivienda
) VALUES (
    :id, :edad, :grado, :estado_civil, :tam_fam,
    :categoria, :ocupacion, :anios_emp, :rend_anual,
    :carro, :vivienda
)
"""

nuevo_cliente = {
    "id": 6850985,
    "edad": 33,
    "grado": "Doctorado",
    "estado_civil": "Soltero",
    "tam_fam": 1,
    "categoria": "Empleado",
    "ocupacion": "TI",
    "anios_emp": 2,
    "rend_anual": 290000,
    "carro": 0,
    "vivienda": "Casa/Departamento propio"
}


try:
    with engine.connect() as conn:
        conn.execute(text(insert_query), nuevo_cliente)
        conn.commit()
        print(" Cliente insertado correctamente.")
except SQLAlchemyError as e:
    print(" Error al insertar:", e)


 Cliente insertado correctamente.


In [53]:
# Volver a cargar la tabla desde SQLite
df_actualizado = pd.read_sql("SELECT * FROM clientes", engine)

# Verificar el cliente nuevo
df_actualizado[df_actualizado["ID_Cliente"] == 6850985]

,ID_Cliente,Edad,Grado_estudio,Estado_civil,Tamaño_familia,Categoria_de_renta,Ocupacion,Años_empleado,Rendimiento_anual,Tiene_carro,Vivienda
438462,6850985,33,Doctorado,Soltero,1,Empleado,TI,2,290000.0,0,Casa/Departamento propio
